In [2]:
import numpy as np
import pandas as pd
import keras
import eng_to_ipa as ipa

Using TensorFlow backend.


In [3]:
from nltk.tokenize import PunktSentenceTokenizer
from nltk import word_tokenize
from nltk import pos_tag

In [4]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [5]:
poems = df['Poem'][df.Form == 'Sonnet']
poems

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
                            ...                        
95    Whate'er the cost to me, with this farewell,\n...
96    If the past year were offered me again,\nWith ...
97    Nay, dear one, ask me not to leave thee yet.\n...
98    Where is the pride for which I once was blamed...
99    I sue thee not for pity on my case.\nIf I have...
Name: Poem, Length: 348, dtype: object

In [6]:
text = ''
for poem in poems:
    text += str(poem) + '\n'
len(text)

258640

In [7]:
text = text.replace("’", "'")
text = text.replace("‘", "'")
text = text.replace("“", " ")
text = text.replace("'d", "ed")
text = text.replace("(", "")
text = text.replace(")", "")
text = text.replace(" '", " ")
text = text.replace("' ", " ")
text = text.replace('"', '')
text = text.replace("--", " ")
text = text.replace(":-", " ")
text = text.replace("-:", " ")
text = text.replace(".-", " ")
text = text.replace("- ", " ")
text = text.replace(" -", " ")
text = text.replace(" –", " ")
while text.find(" . ") != -1:
    text = text.replace(" . ", " ")
while text.find("..") != -1:
    text = text.replace("..", ".")
text = text.replace("?.", "? ")
text = text.replace("!.", "! ")
text = text.replace("!-", "! ")
text = text.replace("!—", "! ")
text = text.replace(".", " ")
text = text.replace(",", " ")
# text = text.replace("\n"," eol\n")
text = text.replace("\r"," ")

In [20]:
# train the Punkt tokenizer
custom_sent_tokenizer = PunktSentenceTokenizer(text)

In [21]:
# tokenize using the sample text
tokenized = custom_sent_tokenizer.tokenize(text)
print(len(tokenized))

511


In [22]:
# tokenized[0]

In [23]:
try:
    tagged = []
    for i in tokenized:
        i = i.strip('nan\n')
        words = word_tokenize(i)
        tagged.extend(pos_tag(words))
    print(tagged[0])
except Exception as e:
    print(e)

('From', 'IN')


In [24]:
# text_list = text.split()
# a = 0
# roman_num = ['ii','iii','iv','v','vi','vii','viii','ix','x','xi','xii','xiii','xiv','xv','xvi','xvii',
#              'xviii','xix','xx','xxi','xxii','xxiii','xxiv','xxv','xxvi','xxvii','xxviii','xxix','xxx']
# for i in range(len(text_list) - a):
#     text_list[i - a] = text_list[i - a].strip("“”''-.?:;!,[]~`’—&")
#     if text_list[i - a] in ['nan','','*'] or text_list[i-a][0] == '&' or text_list[i-a].isdigit() \
#      or text_list[i-a] in roman_num:
#         text_list.pop(i - a)
#         a += 1
# len(text_list)

In [25]:
# try:
#     tagged = []
#     for i in text_list:
# #         i = i.strip('nan\n')
#         words = word_tokenize(i)
#         tagged.extend(pos_tag(words))
#     print(tagged[0])
# except Exception as e:
#     print(e)

In [26]:
print(tagged)

[('From', 'IN'), ('fairest', 'JJS'), ('creatures', 'NNS'), ('we', 'PRP'), ('desire', 'VBP'), ('increase', 'VB'), ('That', 'DT'), ('thereby', 'JJ'), ('beauty', 'NN'), ("'s", 'POS'), ('rose', 'VBD'), ('might', 'MD'), ('never', 'RB'), ('die', 'VB'), ('But', 'CC'), ('as', 'IN'), ('the', 'DT'), ('riper', 'NN'), ('should', 'MD'), ('by', 'IN'), ('time', 'NN'), ('decease', 'VB'), ('His', 'PRP$'), ('tender', 'NN'), ('heir', 'NN'), ('might', 'MD'), ('bear', 'VB'), ('his', 'PRP$'), ('memory', 'NN'), (':', ':'), ('But', 'CC'), ('thou', 'RB'), ('contracted', 'VBD'), ('to', 'TO'), ('thine', 'VB'), ('own', 'JJ'), ('bright', 'JJ'), ('eyes', 'NNS'), ("Feed'st", 'NNP'), ('thy', 'VBD'), ('light', 'NN'), ("'s", 'POS'), ('flame', 'NN'), ('with', 'IN'), ('self-substantial', 'JJ'), ('fuel', 'NN'), ('Making', 'VBG'), ('a', 'DT'), ('famine', 'NN'), ('where', 'WRB'), ('abundance', 'NN'), ('lies', 'VBZ'), ('Thyself', 'NNP'), ('thy', 'JJ'), ('foe', 'NN'), ('to', 'TO'), ('thy', 'VB'), ('sweet', 'JJ'), ('self', 'NN

In [27]:
tagged[37:39]

[('eyes', 'NNS'), ("Feed'st", 'NNP')]

In [28]:
maxlen = 6                                                            
step = 1 
word_list = list(tagged)

sentences = []                                                         

next_words = []                                                        

for i in range(0, len(word_list) - maxlen, step):
    sentences.append(word_list[i: i + maxlen])
    next_words.append(word_list[i + maxlen])

print('Number of sequences:', len(sentences))

words = sorted(list(set(word_list)))                                        
print('Unique words:', len(words))
word_indices = dict((word, words.index(word)) for word in words)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)      
y = np.zeros((len(sentences), len(words)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, word in enumerate(sentence):                                
        x[i, t, word_indices[word]] = 1                                
    y[i, word_indices[next_words[i]]] = 1 

Number of sequences: 48352
Unique words: 9530
Vectorization...


In [29]:
word_indices

{('!', '.'): 0,
 ("'", "''"): 1,
 ("'", 'POS'): 2,
 ("'Cause", 'CC'): 3,
 ("'Neath", 'POS'): 4,
 ("'Sweet", 'CD'): 5,
 ("'Tis", "''"): 6,
 ("'Tis", 'CD'): 7,
 ("'Tis", 'NN'): 8,
 ("'Tis", 'VB'): 9,
 ("'Tis", 'VBZ'): 10,
 ("'Twas", 'CD'): 11,
 ("'You", "''"): 12,
 ("'and", 'CD'): 13,
 ("'and", 'PRP'): 14,
 ("'ll", 'MD'): 15,
 ("'re", 'VBP'): 16,
 ("'s", 'POS'): 17,
 ("'s", 'VBZ'): 18,
 ("'t", "''"): 19,
 ("'untie", "''"): 20,
 ("'ve", 'VBP'): 21,
 ("'~Proverb", 'VBZ'): 22,
 ('-will', 'NNP'): 23,
 ('0', 'CD'): 24,
 ('1', 'CD'): 25,
 ("1'ora", 'CD'): 26,
 ('10', 'CD'): 27,
 ('11', 'CD'): 28,
 ('12', 'CD'): 29,
 ('13', 'CD'): 30,
 ('14', 'CD'): 31,
 ('1927', 'CD'): 32,
 ('2', 'CD'): 33,
 ('23', 'CD'): 34,
 ('26', 'CD'): 35,
 ('3', 'CD'): 36,
 ('4', 'CD'): 37,
 ('5', 'CD'): 38,
 ('6', 'CD'): 39,
 ('7', 'CD'): 40,
 ('8', 'CD'): 41,
 ('9', 'CD'): 42,
 (':', ':'): 43,
 (';', ':'): 44,
 ('?', '.'): 45,
 ('A', 'DT'): 46,
 ('A', 'NNP'): 47,
 ('ALL', 'NNP'): 48,
 ('AS', 'IN'): 49,
 ('AT', 'VBP'): 

In [30]:
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\kyle_\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [31]:
# Method to add phonems for each word to 'phonem' dict
def syls(word):
    flag = False
    r_word = word[:]
    while len(r_word) > 0 and flag == False:
        try:
            phonem[word] = d[r_word.lower()]
            flag = True        # if no exception occurs then flag is set indicating phonems added to dict
        except Exception as e:
            r_word = r_word[1:]
    if r_word == '':
        phonem[word] = []

In [32]:
# initializes the phonem dict
phonem = {}
# calls the 'syls' method for each word in the 'words' list
for w in words:
    syls(w[0])
# phonem

In [33]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(256, input_shape=(maxlen, len(words))))
model.add(layers.Dense(len(words), activation='softmax'))

In [34]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [35]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [36]:
def rhyming_words(word):
    if word != '':
        for key in phonem:
            len_key = len(phonem[key])
            for i in range(len(phonem[word])):
                for j in range(len_key):
                    if word == key or len_key == 0:
                        continue
                    elif len(phonem[word]) == 1:
                        if phonem[word][i][-1] == phonem[key][j][-1]:
                            rhymers.append(key)
                    elif len(phonem[key]) == 2:
                        if phonem[word][i][-2:] == phonem[key][j][-2:]:
                            rhymers.append(key)
                    else:
                        if phonem[word][i][-3:] == phonem[key][j][-3:]:
                            rhymers.append(key)
#     print(len(rhymers))
    
                                           
def syllable_counter(word):
    count = 0
    if word == '' or word == '\n':
        count = 0
    elif ipa.syllable_count(word) == 0:
        count = len(word) % 3
        if count == 0:
            count = 1
    else:
        count += ipa.syllable_count(word)
    return count

In [70]:
import random
import sys

RHYME_CHECK = 5
line_count = 0
quatrain_count = 0
quatrain = False
new_line = False
file = open("poem_output.txt", 'a')
app_word = ''
end_of_line = False

generated_text = ''
count = 0

for epoch in range(1, 11):
    generated_text = ''
    generated_list = []
    print('\nepoch', epoch)
    hist = model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(word_list) - maxlen - 1)         
    generated_list = word_list[start_index: start_index + maxlen]
    print(generated_list)

    for word in generated_list:
        generated_text += word[0] + ' '
    generated_text =  generated_text.strip()
    print('--- Generating with seed: "' + generated_text + '"')
    file.write('--- Generating with seed: "' + generated_text + '"')
    for num in ipa.syllable_count(generated_text):
        count += num
    for temperature in [0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        file.write('------ temperature: ' + str(temperature))
        sys.stdout.write(generated_text)
        file.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(words)))             
            for t, word in enumerate(generated_list): 
#                 print('WORD: ', word)
                sampled[0, t, word_indices[word]] = 1. 
                
            if count >= 10:
                end_of_line = True
#                 sys.stdout.write('\n')
#                 file.write('\n')
                count = 0 
#                 new_line = True

            if line_count == 2 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z1
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if count + syllable_counter(next_word[0]) >= 10:
#                     sys.stdout.write(' THIRD LINE')
#                     print(' WORD:', next_word[0], end='')
                    line_count += 1 
#                     print(f' LC {line_count} QC {quatrain_count} COUNT {count}', end='')
                    if quatrain_count == 3:
                        line_count += 1
                        quatrain_count = 0
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word[0])) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 
               
            elif line_count == 3 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z2
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if (count + syllable_counter(next_word[0]) >= 10):
#                     sys.stdout.write('     FOURTH LINE')
                    line_count += 1
                    quatrain_count += 1
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word[0])) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 

            else:
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                
            if count < 10 and next_word[0] == 'eol':
                next_word = ('','E')
                app_word = generated_list[maxlen - 1]
            elif end_of_line:
                app_word = next_word = ('eol','NN')
                end_of_line = False
                count = 0
            else:
                app_word = next_word[:]

            generated_list.append(app_word)
            generated_list = generated_list[1:]

            if new_line and next_word[0] == 'eol':
                next_word = ('','E')
            elif new_line:
                word_only = next_word[0].capitalize()
                next_word = (word_only,next_word[1])
                new_line = False
            elif next_word == 'i':
                word_only = next_word[0].upper()
                next_word = (word_only,next_word[1])
            elif next_word[0] == 'eol':
                next_word = ('\n','NL')
                new_line = True
            if next_word[0] != '':    
                sys.stdout.write(' ' + next_word[0])
                file.write(' ' + next_word[0])
                syl_num = syllable_counter(next_word[0])
                count += syl_num
                               

            if count >= 10:
#                 print('LINE COUNT: ', line_count)
                if line_count == 0:
#                     sys.stdout.write('     FIRST LINE')
                    rhymers = []
                    z1 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word[0])
                    r_word = next_word[0].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(' R WORD: ', r_word)
                    rhyming_words(r_word)
#                     print(' RHYMERS: ',rhymers)
                    for rhyme in rhymers:
#                         print('\nRHYME: ', rhyme)
                        for tup in word_indices:
#                             print('TUP WORD: ', tup[0])
                            if rhyme == tup[0]:
#                                 print('TUP: ', tup)
                                z1[word_indices[tup]] = 10000.
                    line_count += 1
                    if quatrain_count == 3:
                        line_count += 1

                elif line_count == 1:
#                     sys.stdout.write('  SECOND LINE')
                    rhymers = []
                    z2 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word[0])
                    r_word = next_word[0].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(r_word)
                    rhyming_words(r_word)
#                     print('\nRHYMERS: ', rhymers)
                    for rhyme in rhymers:
#                         print('\nRHYME: ', rhyme)
                        for tup in word_indices:
                            if rhyme == tup[0]:
#                                 print('TUP: ', tup)
                                z1[word_indices[tup]] = 10000.
                    line_count += 1
#                     print('     LINE COUNT:', line_count, end = '')

                elif line_count == 4:
                    sys.stdout.write('\n')
                    file.write('\n')
                    line_count = 0
file.close()


epoch 1
Epoch 1/1
52745/52745 [==============================] - 162s 3ms/step - loss: 1.9850
[('do', 'VB'), ('contend', 'NN'), ('eol', 'NN'), ('nativity', 'NN'), ('once', 'RB'), ('in', 'IN')]
--- Generating with seed: "do contend eol nativity once in"
------ temperature: 0.5
do contend eol nativity once in the 
 Of light was stay and my love 's bright are 
 Thy might with much to be grief more dost or 
 To be and why with love you can not be
 
 To good but all my and her she is my 
 And she will you right so my love be heart 
 O thou with me and you will one love most 
 Eyes knows not thy simple with speak 's heart
 
 And grow who in this with me is with my 
 o ye all will that making i what to 
 Hath the last year reach can be is my die 
 But if the flower shall we yourself among
 
 Was shall this against my is lies where all 
 O here death my belovëd who i out thee i
 
 Then on the painting must and age there to 
 But half the spring to in the chaste change thy 
 And for me love al

KeyError: 's'

In [ ]:
hist.history

In [ ]:


hist.history

In [ ]:
# from gtts import gTTS
# import os
# language = 'en'
# myobj = gTTS(text=generated_text, lang=language, slow=False)
# myobj.save('welcome.mp3')
# os.system('welcome.mp3')


In [ ]:
a = 'black'
b = 'blackly'
if a in b:
    print('OK')